# Variational AutoEncoder [VAE]

Sources: Kingma D. P., Welling M.: Auto-Encoding Variational Bayes

**General Problem Description**

Let's say we observe values $x_i$ of $N$ independent identically distributed random variables $X_i, i=1, \dots, N$. the values are generated in the following manner: first we generate a value $\theta$ from some distribution $P_{\Theta}$ and then we generate $N$ samples $z_i$ from some distribution $P_{Z}$. After that we generate $x_i$ as a sample from distribution $P_{X \mid \theta, z_i}.$ All the samples from $P_Z$ are generated independently and are all independent with the sample from $P_{\Theta}. $We know $P_{Z}$, $P_{X \mid \theta, z}$ and $P_{\Theta}.$

**General Problem Solution**

Let's now deal with the case $N = 1, x_1 = x^{obs}$.

Now bayesian would ask for the $P_{\Theta, Z | X}$ or at least for $P_{\Theta, Z | x^{obs}}$. Imagine that in our case the relationship between $X$ and $\theta, Z$ is given by some complicated function like a neural network so classical inference is not possible and we have to use variational method. 

Let's parametrize $P_{\Theta, Z \mid X}(\theta, z \mid x)$ as $\delta_{\tilde{\theta}}(\theta) q_{\phi}(z \mid x)$, where $\delta_{\tilde{\theta}}(\theta)$ is $1$ only for $\theta = \tilde{\theta}$ and $0$ otherwise (this is strictly speaking not exactly correct). Note that $q_{\phi}(z \mid x)$ is still only distribution on $z$, it is some function which gives us the family of distribution we would assume on $z$ if we observe some $x$.

If we plug this into the method described in notebook `Variational Methods.ipynb` as $Q_{\eta}(z)$ (in our case $\eta = (\tilde{\theta}, \phi)$ and $z = (\theta, z)$), equation


\begin{equation}
D_{KL}\big( Q_{\eta}(z) \parallel P_{Z \mid X}(z \mid x) \big) = \mathrm{log} \big ( P_{X}(x^{obs}) \big ) - \mathcal{L}(Q_{\eta})
\end{equation}

rewrites as

\begin{equation}
\int \delta_{\tilde{\theta}}(\theta) q_{\phi}(z \mid x^{obs}) \mathrm{log} \bigg ( \frac{\delta_{\tilde{\theta}}(\theta) q_{\phi}(z\mid x^{obs})}{P_{\Theta, Z \mid X}(\theta, z \mid x^{obs})} \bigg ) d\theta dz = \mathrm{log}(P_{X}(x^{obs})) - -\int \delta_{\tilde{\theta}}(\theta) q_{\phi}(z\mid x^{obs}) \mathrm{log} \bigg ( \frac{\delta_{\tilde{\theta}}(\theta) q_{\phi}(z\mid x^{obs})}{P_{\Theta, Z, X}(\theta, z,  x^{obs})} \bigg ) d\theta dz
\end{equation}

So after integrating $d\theta$ we have 

\begin{equation}
\mathcal{L} = -\int q_{\phi}(z\mid x^{obs}) \mathrm{log} \bigg ( \frac{q_{\phi}(z\mid x^{obs})}{P_{\Theta, Z, X}(\tilde{\theta}, z,  x^{obs})} \bigg )dz
\end{equation}


Our goal is to maximize $\mathcal{L}$ with respect to $\tilde{\theta}, \phi$

Now for the general case $N > 1$ the only difference is that we would maximize $\sum_{i=1}^{N} \mathcal{L}(\theta, \phi, x_i)$

For the optimization we would like to calculate gradients of $\mathcal{\theta, \phi, x_i}$ with respect to $\theta, \phi$. There are general methods to do this. Kingma proposes method with lower variance for some special cases, which are very common in practice. 

We will follow the notation in Kingma. 

Note that $\mathcal{L} = -D_{KL}\big(q_{\phi}(z \mid x) \parallel p_{\theta}(z) \big) - E_{q_{\phi}(z \mid x)}\mathrm{log} p_{\theta}(x \mid z)$. We will use this fact in the next section.

**Variational AutoEncoder Setting**

Set: 
- $\mathrm{dim}(z) = J$
- $p_{\theta}(z) = N(0, I)$ is multivariate Gaussian
- $p_{\theta}(x \mid z)$ will be multivariate Bernoulli / Gaussian whose parameters are computed from $z$ using multi-layer perceptron 
- $p_{\phi}(z \mid x) = N(\mu, \sigma^{2}I)$ is multivariate Gaussian, where the vectors $\mu$ and $\sigma$ are calculated as multi-layer perceptron from $x, \phi$

Now using 2 tricks Kingma introduces -- the fact that in this case $D_{KL}$ is tractable and $\mathrm{log} p_{\theta(x \mid z)}$ can be sampled using his reparametrization trick, we get: 

\begin{equation}
\mathcal{L}(\theta, \phi, x) \simeq \frac{1}{2}\sum_{j = 1}^{J} \bigg ( 1 + \mathrm{log}(\sigma_j^2) - \mu_j^2 - \sigma_j^2 \bigg) + \frac{1}{L} \sum_{l = 1}^{L} \mathrm{log} p_{\theta}(x \mid z^{(j)})
\end{equation}

where $z^{(1)}, \dots, z^{(L)}$ is sampled from $N(\mu, \sigma^{2}I)$


**Exercise**

Calculate the explicit form of $-D_{KL}$ in the setting of variational autoencoder. 

**Solution**

\begin{equation}
-D_{KL} = - \int \frac{1}{\sqrt{(2\pi)^J \prod_{j=1}^{J} \sigma_{j}^{2}}} e^{-\frac{1}{2}\sum \big ( \frac{z_{j}-\mu_{j}}{\sigma_{j}} \big )^{2}} \mathrm{log} \Bigg \lbrace \frac{\frac{1}{\sqrt{(2\pi)^J \prod_{j=1}^{J}\sigma_{j}^{2}}} e^{-\frac{1}{2}\sum \big ( \frac{z_{j}-\mu_{j}}{\sigma_{j}} \big )^{2}}}{\frac{1}{\sqrt{(2\pi)^J}} e^{-\frac{1}{2}\sum_{j=1}^{J} z_{j}^{2}}} \Bigg \rbrace dz
\end{equation}

\begin{equation} 
= \frac{1}{2} \int q_{\phi}(z \mid x) \bigg \lbrace \sum_{j=1}^{J} \mathrm{log} \sigma_{j}^2 + \sum_{j=1}^{J} \big ( \frac{z_{j}-\mu_{j}}{\sigma_{j}^{2}} \big )^{2} - \sum_{j=1}^{J} z_{j}^{2} \bigg  \rbrace dz
\end{equation}

which gives the desired result after realizing that $\int q_{\phi}(z \mid x) \big( \frac{z_{j}-\mu_{j}}{\sigma_{j}^{2}} \big )^{2} dz_{j} = 1$ (and similarly for the integral from $z_{j}^{2}$).
